---
title: Assignment 04
author:
  - name: Joshua Lawrence
    affiliations:
      - id: bu
        name: Boston University
        city: Boston
        state: MA
number-sections: true
date: '2025-10-08'
date-modified: today
date-format: long
format:
  html:
    theme: sandstone
    toc: true
    toc-depth: 2
  docx: default
execute:
  echo: true
  eval: true
  freeze: auto
---

# Load the Dataset

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.regression import LinearRegression, RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator
import pandas as pd
import plotly.express as px
import plotly.io as pio
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math

# Set random seed for reproducibility
np.random.seed(42)
pio.renderers.default = "notebook+notebook_connected+vscode"

# Initialize Spark Session
spark = SparkSession.builder.appName("LightcastData").getOrCreate()

# Load Data
df = spark.read.option("header", "true").option("inferSchema", "true").option("multiLine","true").option("escape", "\"").csv("data/lightcast_job_postings.csv")

# Show Schema and Sample Data
# df.printSchema() # comment this line when rendering the submission
df.show(5)

# Feature Engineering

In [ ]:
# Define features and target variable
continuous_features = ['MIN_YEARS_EXPERIENCE', 'MAX_YEARS_EXPERIENCE', 'SALARY_FROM', 'SALARY_TO']
categorical_features = ['REMOTE_TYPE_NAME', 'EMPLOYMENT_TYPE_NAME']
target_variable = 'SALARY'

# Select columns and drop missing values
selected_columns = continuous_features + categorical_features + [target_variable]
df_cleaned = df.select(*selected_columns).na.drop()

# Create StringIndexers for categorical variables
indexers = [
    StringIndexer(inputCol=col_name, outputCol=col_name + "_index", handleInvalid="keep")
    for col_name in categorical_features
]

# Create OneHotEncoders for indexed categorical variables
encoders = [
    OneHotEncoder(inputCol=col_name + "_index", outputCol=col_name + "_encoded")
    for col_name in categorical_features
]

# Prepare feature column names
encoded_categorical_cols = [col_name + "_encoded" for col_name in categorical_features]
feature_cols = continuous_features + encoded_categorical_cols

# Create VectorAssembler for linear features
assembler = VectorAssembler(
    inputCols=feature_cols,
    outputCol="features",
    handleInvalid="skip"
)

# Build pipeline
pipeline_stages = indexers + encoders + [assembler]
pipeline = Pipeline(stages=pipeline_stages)

# Fit and transform data
df_transformed = pipeline.fit(df_cleaned).transform(df_cleaned)

# Create polynomial feature: MIN_YEARS_EXPERIENCE_SQ
df_poly = df_transformed.withColumn(
    "MIN_YEARS_EXPERIENCE_SQ", 
    col("MIN_YEARS_EXPERIENCE") * col("MIN_YEARS_EXPERIENCE")
)

# Create VectorAssembler for polynomial features
poly_feature_cols = continuous_features + ["MIN_YEARS_EXPERIENCE_SQ"] + encoded_categorical_cols
assembler_poly = VectorAssembler(
    inputCols=poly_feature_cols,
    outputCol="features_poly",
    handleInvalid="skip"
)

# Transform data with polynomial features
df_final = assembler_poly.transform(df_poly)

# Train/Test Split

In [ ]:
# Split data into training and testing sets (80/20 split)
train_final, test_final = df_final.randomSplit([0.8, 0.2], seed=42)

# Get counts
train_count = train_final.count()
test_count = test_final.count()
total_count = df_final.count()

print(f"Total observations: {total_count}")
print(f"Training set: {train_count} ({train_count/total_count*100:.1f}%)")
print(f"Testing set: {test_count} ({test_count/total_count*100:.1f}%)")

# Display sample data
print("\nTraining Data Sample:")
train_final.select("SALARY", "MIN_YEARS_EXPERIENCE", "MAX_YEARS_EXPERIENCE", "SALARY_FROM", "SALARY_TO", "REMOTE_TYPE_NAME", "EMPLOYMENT_TYPE_NAME").show(5)

print("\nTest Data Sample:")
test_final.select("SALARY", "MIN_YEARS_EXPERIENCE", "MAX_YEARS_EXPERIENCE", "SALARY_FROM", "SALARY_TO", "REMOTE_TYPE_NAME", "EMPLOYMENT_TYPE_NAME").show(5)

# Linear Regression

In [ ]:
# Initialize Linear Regression model
lr = LinearRegression(featuresCol="features", labelCol="SALARY", maxIter=100, regParam=0.1, elasticNetParam=0.0)

# Train the model
lr_model = lr.fit(train_final)

# Generate predictions
train_predictions = lr_model.transform(train_final)
test_predictions = lr_model.transform(test_final)

# Initialize evaluators
evaluator_r2 = RegressionEvaluator(labelCol="SALARY", predictionCol="prediction", metricName="r2")
evaluator_rmse = RegressionEvaluator(labelCol="SALARY", predictionCol="prediction", metricName="rmse")
evaluator_mae = RegressionEvaluator(labelCol="SALARY", predictionCol="prediction", metricName="mae")

# Evaluate on training set
train_r2 = evaluator_r2.evaluate(train_predictions)
train_rmse = evaluator_rmse.evaluate(train_predictions)
train_mae = evaluator_mae.evaluate(train_predictions)

# Evaluate on test set
test_r2 = evaluator_r2.evaluate(test_predictions)
test_rmse = evaluator_rmse.evaluate(test_predictions)
test_mae = evaluator_mae.evaluate(test_predictions)

# Extract model parameters
coefficients = lr_model.coefficients
intercept = lr_model.intercept
summary = lr_model.summary

# Prepare feature names
feature_names = continuous_features + encoded_categorical_cols

# Extract coefficient statistics
try:
    coef_std_errors = summary.coefficientStandardErrors
    t_values = summary.tValues
    p_values = summary.pValues
    
    coef_data = []
    for i, feature_name in enumerate(feature_names):
        if i < len(coefficients):
            coef = coefficients[i]
            std_err = coef_std_errors[i]
            t_val = t_values[i]
            p_val = p_values[i]
            ci_lower = coef - 1.96 * std_err
            ci_upper = coef + 1.96 * std_err
            
            coef_data.append({
                'Feature': feature_name,
                'Coefficient': coef,
                'Std_Error': std_err,
                'T_Value': t_val,
                'P_Value': p_val,
                'CI_Lower': ci_lower,
                'CI_Upper': ci_upper
            })
    
    coef_df = pd.DataFrame(coef_data)
    
    intercept_std_err = coef_std_errors[-1]
    intercept_t_val = t_values[-1]
    intercept_p_val = p_values[-1]
    intercept_ci_lower = intercept - 1.96 * intercept_std_err
    intercept_ci_upper = intercept + 1.96 * intercept_std_err
    
except:
    coef_data = []
    for i, feature_name in enumerate(feature_names):
        if i < len(coefficients):
            coef = coefficients[i]
            
            coef_data.append({
                'Feature': feature_name,
                'Coefficient': coef,
                'Std_Error': 'N/A',
                'T_Value': 'N/A',
                'P_Value': 'N/A',
                'CI_Lower': 'N/A',
                'CI_Upper': 'N/A'
            })
    
    coef_df = pd.DataFrame(coef_data)
    
    intercept_std_err = 'N/A'
    intercept_t_val = 'N/A'
    intercept_p_val = 'N/A'
    intercept_ci_lower = 'N/A'
    intercept_ci_upper = 'N/A'

# Print results
print("=" * 80)
print("LINEAR REGRESSION MODEL SUMMARY")
print("=" * 80)

print("\n--- Model Performance Metrics ---")
print(f"Intercept: {intercept:,.2f}")
print(f"\nTraining Set Performance:")
print(f"  R² Score: {train_r2:.4f}")
print(f"  RMSE: ${train_rmse:,.2f}")
print(f"  MAE: ${train_mae:,.2f}")
print(f"\nTest Set Performance:")
print(f"  R² Score: {test_r2:.4f}")
print(f"  RMSE: ${test_rmse:,.2f}")
print(f"  MAE: ${test_mae:,.2f}")

print("\n--- Feature Coefficients and Statistical Significance ---")
print(coef_df.to_string(index=False))

print("\n--- Intercept Statistics ---")
print(f"Intercept: {intercept:,.2f}")
print(f"Std Error: {intercept_std_err}")
print(f"T-Value: {intercept_t_val}")
print(f"P-Value: {intercept_p_val}")
print(f"95% CI: [{intercept_ci_lower}, {intercept_ci_upper}]")

print("\n" + "=" * 80)
print("INTERPRETATION OF RESULTS")
print("=" * 80)

print("\n1. MODEL PERFORMANCE INTERPRETATION:")
print(f"   - R² Score ({test_r2:.4f}): Indicates that {test_r2*100:.2f}% of the variance in SALARY")
print("     is explained by the model. A higher R² suggests a better model fit.")
print(f"   - RMSE (${test_rmse:,.2f}): On average, predictions deviate by ${test_rmse:,.2f} from")
print("     actual salaries. Lower RMSE indicates better prediction accuracy.")
print(f"   - MAE (${test_mae:,.2f}): The average absolute error is ${test_mae:,.2f}.")
print("     This is more interpretable and less sensitive to outliers than RMSE.")

print("\n2. COEFFICIENT INTERPRETATION:")
print("   - Each coefficient represents the change in SALARY for a one-unit increase")
print("     in that feature, holding all other features constant.")
print("   - Continuous features (MIN_YEARS_EXPERIENCE, MAX_YEARS_EXPERIENCE, SALARY_FROM, SALARY_TO):")
print("     * Positive coefficients: Higher values increase predicted salary")
print("     * Negative coefficients: Higher values decrease predicted salary")
print("   - Encoded categorical features (REMOTE_TYPE_NAME, EMPLOYMENT_TYPE_NAME):")
print("     * Each encoded category represents a binary indicator (0 or 1)")
print("     * Positive coefficients: That category increases salary vs. baseline")
print("     * Negative coefficients: That category decreases salary vs. baseline")

print("\n3. STATISTICAL SIGNIFICANCE:")
print("   - P-Value < 0.05: Feature is statistically significant (reject null hypothesis)")
print("   - P-Value > 0.05: Feature may not be statistically significant")
print("   - T-Value: Larger absolute values indicate stronger evidence against null hypothesis")
print("   - 95% Confidence Interval: If it doesn't contain 0, the feature is significant")

significant_features = coef_df[pd.to_numeric(coef_df['P_Value'], errors='coerce') < 0.05]
print(f"\n   Statistically significant features (p < 0.05): {len(significant_features)}/{len(coef_df)}")

print("\n4. KEY INSIGHTS:")
print("   - MULTICOLLINEARITY ISSUE: SALARY_FROM and SALARY_TO essentially contain")
print("     the information we're trying to predict (SALARY). This is called data leakage.")
print("   - In a real-world scenario, these features would not be available at prediction time.")
print("   - This explains the high R² but also why statistical inference is problematic.")
print("   - RECOMMENDATION: Remove SALARY_FROM and SALARY_TO, or create derived features")
print("     like salary_range = SALARY_TO - SALARY_FROM for more realistic modeling.")

print("\n5. FEATURE IMPORTANCE (by absolute coefficient value):")
coef_abs = coef_df.copy()
if coef_abs['Coefficient'].dtype in ['float64', 'int64']:
    coef_abs['Abs_Coefficient'] = abs(coef_abs['Coefficient'])
    coef_abs_sorted = coef_abs.sort_values('Abs_Coefficient', ascending=False)
    print(coef_abs_sorted[['Feature', 'Coefficient', 'Abs_Coefficient']].head(10).to_string(index=False))

print("\n" + "=" * 80)

# Show sample predictions
test_predictions.select("SALARY", "prediction", "features").show(10, truncate=False)

# Polynomial Regression

In [ ]:
# Initialize Polynomial Regression model
lr_poly = LinearRegression(featuresCol="features_poly", labelCol="SALARY", maxIter=100, regParam=0.1, elasticNetParam=0.0)

# Train the model
lr_poly_model = lr_poly.fit(train_final)

# Generate predictions
train_predictions_poly = lr_poly_model.transform(train_final)
test_predictions_poly = lr_poly_model.transform(test_final)

# Evaluate on training set
train_r2_poly = evaluator_r2.evaluate(train_predictions_poly)
train_rmse_poly = evaluator_rmse.evaluate(train_predictions_poly)
train_mae_poly = evaluator_mae.evaluate(train_predictions_poly)

# Evaluate on test set
test_r2_poly = evaluator_r2.evaluate(test_predictions_poly)
test_rmse_poly = evaluator_rmse.evaluate(test_predictions_poly)
test_mae_poly = evaluator_mae.evaluate(test_predictions_poly)

# Extract model parameters
coefficients_poly = lr_poly_model.coefficients
intercept_poly = lr_poly_model.intercept
summary_poly = lr_poly_model.summary

# Prepare polynomial feature names
poly_feature_names = continuous_features + ["MIN_YEARS_EXPERIENCE_SQ"] + encoded_categorical_cols

# Extract coefficient statistics
try:
    coef_std_errors_poly = summary_poly.coefficientStandardErrors
    t_values_poly = summary_poly.tValues
    p_values_poly = summary_poly.pValues
    
    coef_data_poly = []
    for i, feature_name in enumerate(poly_feature_names):
        if i < len(coefficients_poly):
            coef = coefficients_poly[i]
            std_err = coef_std_errors_poly[i]
            t_val = t_values_poly[i]
            p_val = p_values_poly[i]
            ci_lower = coef - 1.96 * std_err
            ci_upper = coef + 1.96 * std_err
            
            coef_data_poly.append({
                'Feature': feature_name,
                'Coefficient': coef,
                'Std_Error': std_err,
                'T_Value': t_val,
                'P_Value': p_val,
                'CI_Lower': ci_lower,
                'CI_Upper': ci_upper
            })
    
    coef_df_poly = pd.DataFrame(coef_data_poly)
    
    intercept_std_err_poly = coef_std_errors_poly[-1]
    intercept_t_val_poly = t_values_poly[-1]
    intercept_p_val_poly = p_values_poly[-1]
    intercept_ci_lower_poly = intercept_poly - 1.96 * intercept_std_err_poly
    intercept_ci_upper_poly = intercept_poly + 1.96 * intercept_std_err_poly
    
except:
    coef_data_poly = []
    for i, feature_name in enumerate(poly_feature_names):
        if i < len(coefficients_poly):
            coef = coefficients_poly[i]
            
            coef_data_poly.append({
                'Feature': feature_name,
                'Coefficient': coef,
                'Std_Error': 'N/A',
                'T_Value': 'N/A',
                'P_Value': 'N/A',
                'CI_Lower': 'N/A',
                'CI_Upper': 'N/A'
            })
    
    coef_df_poly = pd.DataFrame(coef_data_poly)
    
    intercept_std_err_poly = 'N/A'
    intercept_t_val_poly = 'N/A'
    intercept_p_val_poly = 'N/A'
    intercept_ci_lower_poly = 'N/A'
    intercept_ci_upper_poly = 'N/A'

# Print results
print("\n" + "=" * 80)
print("POLYNOMIAL REGRESSION MODEL SUMMARY")
print("=" * 80)

print("\n--- Model Performance Metrics ---")
print(f"Intercept: {intercept_poly:,.2f}")
print(f"\nTraining Set Performance:")
print(f"  R² Score: {train_r2_poly:.4f}")
print(f"  RMSE: ${train_rmse_poly:,.2f}")
print(f"  MAE: ${train_mae_poly:,.2f}")
print(f"\nTest Set Performance:")
print(f"  R² Score: {test_r2_poly:.4f}")
print(f"  RMSE: ${test_rmse_poly:,.2f}")
print(f"  MAE: ${test_mae_poly:,.2f}")

print("\n--- Feature Coefficients and Statistical Significance ---")
print(coef_df_poly.to_string(index=False))

print("\n--- Intercept Statistics ---")
print(f"Intercept: {intercept_poly:,.2f}")
print(f"Std Error: {intercept_std_err_poly}")
print(f"T-Value: {intercept_t_val_poly}")
print(f"P-Value: {intercept_p_val_poly}")
print(f"95% CI: [{intercept_ci_lower_poly}, {intercept_ci_upper_poly}]")

print("\n" + "=" * 80)
print("COMPARISON: LINEAR vs POLYNOMIAL REGRESSION")
print("=" * 80)
print(f"\nLinear Regression Test R²: {test_r2:.4f}")
print(f"Polynomial Regression Test R²: {test_r2_poly:.4f}")
print(f"Improvement in R²: {test_r2_poly - test_r2:.4f}")

print(f"\nLinear Regression Test RMSE: ${test_rmse:,.2f}")
print(f"Polynomial Regression Test RMSE: ${test_rmse_poly:,.2f}")
print(f"Change in RMSE: ${test_rmse_poly - test_rmse:,.2f}")

print(f"\nLinear Regression Test MAE: ${test_mae:,.2f}")
print(f"Polynomial Regression Test MAE: ${test_mae_poly:,.2f}")
print(f"Change in MAE: ${test_mae_poly - test_mae:,.2f}")

print("\n" + "=" * 80)
print("INTERPRETATION OF POLYNOMIAL REGRESSION RESULTS")
print("=" * 80)

print("\n1. MODEL PERFORMANCE INTERPRETATION:")
print(f"   - R² Score ({test_r2_poly:.4f}): The polynomial model explains {test_r2_poly*100:.2f}% of")
print("     the variance in SALARY, compared to {:.2f}% for the linear model.".format(test_r2*100))
if test_r2_poly > test_r2:
    print(f"     * IMPROVEMENT: The polynomial model captures {(test_r2_poly - test_r2)*100:.2f}% more variance.")
    print("     * This suggests non-linear relationships exist in the data.")
else:
    print("     * The polynomial model does not improve over the linear model.")
    print("     * This suggests the relationship may be primarily linear.")

print(f"\n   - RMSE (${test_rmse_poly:,.2f}): Average prediction error for polynomial model.")
if test_rmse_poly < test_rmse:
    print(f"     * IMPROVEMENT: ${test_rmse - test_rmse_poly:,.2f} reduction in RMSE.")
    print("     * The polynomial model makes more accurate predictions.")
else:
    print(f"     * The polynomial model has higher RMSE by ${test_rmse_poly - test_rmse:,.2f}.")
    print("     * This may indicate overfitting or that polynomial terms don't help.")

print(f"\n   - MAE (${test_mae_poly:,.2f}): Average absolute error for polynomial model.")
if test_mae_poly < test_mae:
    print(f"     * IMPROVEMENT: ${test_mae - test_mae_poly:,.2f} reduction in MAE.")
else:
    print(f"     * The polynomial model has higher MAE by ${test_mae_poly - test_mae:,.2f}.")

if train_r2_poly - test_r2_poly > 0.1:
    print(f"\n   - OVERFITTING WARNING: Training R² ({train_r2_poly:.4f}) is significantly")
    print(f"     higher than test R² ({test_r2_poly:.4f}).")
    print("     * The model may have learned training data patterns that don't generalize.")
    print("     * Consider reducing model complexity or increasing regularization.")
else:
    print(f"\n   - GOOD GENERALIZATION: Training R² ({train_r2_poly:.4f}) and test R²")
    print(f"     ({test_r2_poly:.4f}) are similar, indicating the model generalizes well.")

print("\n2. POLYNOMIAL FEATURE INTERPRETATION:")
print("   - MIN_YEARS_EXPERIENCE_SQ: This squared term captures non-linear relationships")
print("     between experience and salary.")
print("     * Positive coefficient: Salary increases at an accelerating rate with experience")
print("       (e.g., senior positions command exponentially higher salaries)")
print("     * Negative coefficient: Salary increases at a decelerating rate with experience")
print("       (e.g., diminishing returns to additional years of experience)")
print("     * The combination of MIN_YEARS_EXPERIENCE and MIN_YEARS_EXPERIENCE_SQ allows")
print("       the model to fit a parabolic (curved) relationship.")

print("\n3. MODEL SELECTION RECOMMENDATION:")
if test_r2_poly > test_r2 and test_rmse_poly < test_rmse:
    print("   - The polynomial model outperforms the linear model on both R² and RMSE.")
    print("   - RECOMMENDATION: Use the polynomial model for predictions.")
elif test_r2_poly > test_r2 + 0.01:
    print("   - The polynomial model shows modest improvement in R².")
    print("   - RECOMMENDATION: Consider the polynomial model if interpretability isn't critical.")
else:
    print("   - The polynomial model does not substantially improve performance.")
    print("   - RECOMMENDATION: Prefer the simpler linear model (Occam's Razor).")

print("\n" + "=" * 80)

# Show sample predictions
test_predictions_poly.select("SALARY", "prediction", "features_poly").show(10, truncate=False)

# Random Forest Regressor

In [ ]:
# Initialize Random Forest model
rf = RandomForestRegressor(
    featuresCol="features",
    labelCol="SALARY",
    numTrees=200,
    maxDepth=7,
    seed=42,
    maxBins=32
)

# Train the model
rf_model = rf.fit(train_final)

# Generate predictions
train_predictions_rf = rf_model.transform(train_final)
test_predictions_rf = rf_model.transform(test_final)

# Evaluate on training set
train_r2_rf = evaluator_r2.evaluate(train_predictions_rf)
train_rmse_rf = evaluator_rmse.evaluate(train_predictions_rf)
train_mae_rf = evaluator_mae.evaluate(train_predictions_rf)

# Evaluate on test set
test_r2_rf = evaluator_r2.evaluate(test_predictions_rf)
test_rmse_rf = evaluator_rmse.evaluate(test_predictions_rf)
test_mae_rf = evaluator_mae.evaluate(test_predictions_rf)

# Extract feature importances
feature_importances = rf_model.featureImportances
feature_importance_list = [(feature_names[i], float(feature_importances[i])) 
                           for i in range(len(feature_names))]
feature_importance_df = pd.DataFrame(feature_importance_list, 
                                     columns=['Feature', 'Importance'])
feature_importance_df = feature_importance_df.sort_values('Importance', ascending=False)

# Print results
print("\n" + "=" * 80)
print("RANDOM FOREST REGRESSOR MODEL SUMMARY")
print("=" * 80)

print("\n--- Model Hyperparameters ---")
print(f"Number of Trees: {rf.getNumTrees()}")
print(f"Max Depth: {rf.getMaxDepth()}")
print(f"Max Bins: {rf.getMaxBins()}")
print(f"Seed: {rf.getSeed()}")

print("\n--- Model Performance Metrics ---")
print(f"\nTraining Set Performance:")
print(f"  R² Score: {train_r2_rf:.4f}")
print(f"  RMSE: ${train_rmse_rf:,.2f}")
print(f"  MAE: ${train_mae_rf:,.2f}")
print(f"\nTest Set Performance:")
print(f"  R² Score: {test_r2_rf:.4f}")
print(f"  RMSE: ${test_rmse_rf:,.2f}")
print(f"  MAE: ${test_mae_rf:,.2f}")

print("\n--- Feature Importances ---")
print(feature_importance_df.to_string(index=False))

print("\n" + "=" * 80)
print("INTERPRETATION OF RANDOM FOREST RESULTS")
print("=" * 80)

print("\n1. HYPERPARAMETER SELECTION:")
print(f"   - Number of Trees: {rf.getNumTrees()}")
print("     * More trees generally improve performance but increase training time")
print("     * Random Forest averages predictions across all trees, reducing variance")
print(f"\n   - Max Depth: {rf.getMaxDepth()}")
print("     * Controls how deep each tree can grow")
print("     * Deeper trees = more complex patterns but higher overfitting risk")
print("     * Shallower trees = simpler patterns but may underfit")

print("\n2. MODEL PERFORMANCE:")
if test_r2_rf > test_r2 and test_r2_rf > test_r2_poly:
    print("     * BEST PERFORMER: Outperforms both linear and polynomial models")
elif test_r2_rf > test_r2:
    print("     * Outperforms linear regression")
else:
    print("     * Does not improve over simpler models")

overfitting_gap = train_r2_rf - test_r2_rf
if overfitting_gap > 0.15:
    print(f"\n   - OVERFITTING WARNING: Training R² ({train_r2_rf:.4f}) is {overfitting_gap:.4f}")
    print(f"     higher than test R² ({test_r2_rf:.4f})")
elif overfitting_gap > 0.05:
    print(f"\n   - MINOR OVERFITTING: Training R² ({train_r2_rf:.4f}) is slightly higher")
    print(f"     than test R² ({test_r2_rf:.4f})")
else:
    print(f"\n   - EXCELLENT GENERALIZATION: Training and test R² are very similar")

print("\n3. FEATURE IMPORTANCE:")
top_5_features = feature_importance_df.head(5)
print(f"\n   Top 5 Most Important Features:")
for idx, row in top_5_features.iterrows():
    print(f"     {idx+1}. {row['Feature']}: {row['Importance']:.4f} ({row['Importance']*100:.2f}%)")

print("\n" + "=" * 80)

# Show sample predictions
test_predictions_rf.select("SALARY", "prediction", "features").show(10, truncate=False)

## Feature Importance Plot

In [ ]:
# Create feature importance plot
top_10_importance = feature_importance_df.head(10)

plt.figure(figsize=(10, 8))
sns.barplot(data=top_10_importance, y='Feature', x='Importance', palette='viridis')
plt.xlabel('Importance Score', fontsize=12)
plt.ylabel('Feature', fontsize=12)
plt.title('Top 10 Most Important Features - Random Forest Model', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3, axis='x')
plt.tight_layout()
plt.savefig('output/rf_feature_importance.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n--- Feature Importance Interpretation ---")
print("Random Forest feature importances measure the total reduction in prediction")
print("error contributed by each feature across all trees in the ensemble.")

total_importance_top10 = top_10_importance['Importance'].sum()
print(f"\n- Top 10 features account for {total_importance_top10*100:.1f}% of total importance")

total_importance_top5 = feature_importance_df.head(5)['Importance'].sum()
print(f"- Top 5 features account for {total_importance_top5*100:.1f}% of total importance")

if feature_importance_df.iloc[0]['Importance'] > 0.4:
    print(f"\nWARNING: {feature_importance_df.iloc[0]['Feature']} dominates with")
    print(f"{feature_importance_df.iloc[0]['Importance']*100:.1f}% importance!")
    print("This strongly suggests data leakage.")

# Compare 3 Models – GLR, Polynomial, RF

In [ ]:
# Function to calculate log-likelihood and BIC
def calculate_log_likelihood_and_bic(predictions, n_params):
    predictions_pd = predictions.select("SALARY", "prediction").toPandas()
    residuals = predictions_pd["SALARY"] - predictions_pd["prediction"]
    rss = (residuals ** 2).sum()
    
    dispersion = rss / (len(predictions_pd) - n_params)
    n = len(predictions_pd)
    
    log_likelihood = -0.5 * (n * math.log(2 * math.pi) + n * math.log(dispersion) + rss / dispersion)
    bic = n_params * math.log(n) - 2 * log_likelihood
    
    return log_likelihood, bic

# Calculate number of parameters for each model
n_features_linear = len(feature_names)
n_params_linear = n_features_linear + 1

n_features_poly = len(poly_feature_names)
n_params_poly = n_features_poly + 1

n_params_rf = n_features_linear + 1

# Calculate log-likelihood and BIC for each model
log_likelihood_linear, bic_linear = calculate_log_likelihood_and_bic(test_predictions, n_params_linear)
log_likelihood_poly, bic_poly = calculate_log_likelihood_and_bic(test_predictions_poly, n_params_poly)
log_likelihood_rf, bic_rf = calculate_log_likelihood_and_bic(test_predictions_rf, n_params_rf)

# Calculate AIC
try:
    aic_linear = summary.aic
except:
    aic_linear = 2 * n_params_linear - 2 * log_likelihood_linear

try:
    aic_poly = summary_poly.aic
except:
    aic_poly = 2 * n_params_poly - 2 * log_likelihood_poly

aic_rf = 2 * n_params_rf - 2 * log_likelihood_rf

# Create comprehensive comparison dataframe
comparison_metrics = {
    'Model': ['Linear Regression', 'Polynomial Regression', 'Random Forest'],
    'Test_R2': [test_r2, test_r2_poly, test_r2_rf],
    'Test_RMSE': [test_rmse, test_rmse_poly, test_rmse_rf],
    'Test_MAE': [test_mae, test_mae_poly, test_mae_rf],
    'Train_R2': [train_r2, train_r2_poly, train_r2_rf],
    'Overfitting_Gap': [train_r2 - test_r2, train_r2_poly - test_r2_poly, train_r2_rf - test_r2_rf],
    'AIC': [aic_linear, aic_poly, aic_rf],
    'BIC': [bic_linear, bic_poly, bic_rf],
    'Log_Likelihood': [log_likelihood_linear, log_likelihood_poly, log_likelihood_rf],
    'Num_Parameters': [n_params_linear, n_params_poly, n_params_rf]
}
comparison_df = pd.DataFrame(comparison_metrics)

print("\n" + "=" * 80)
print("COMPREHENSIVE MODEL COMPARISON")
print("=" * 80)
print("\n" + comparison_df.to_string(index=False))

# Identify best performers
best_r2_idx = comparison_df['Test_R2'].idxmax()
best_rmse_idx = comparison_df['Test_RMSE'].idxmin()
best_mae_idx = comparison_df['Test_MAE'].idxmin()
best_aic_idx = comparison_df['AIC'].idxmin()
best_bic_idx = comparison_df['BIC'].idxmin()

print("\n" + "=" * 80)
print("BEST PERFORMERS BY METRIC")
print("=" * 80)
print(f"\nBest R² Score: {comparison_df.iloc[best_r2_idx]['Model']} ({comparison_df.iloc[best_r2_idx]['Test_R2']:.4f})")
print(f"Best RMSE: {comparison_df.iloc[best_rmse_idx]['Model']} (${comparison_df.iloc[best_rmse_idx]['Test_RMSE']:,.2f})")
print(f"Best MAE: {comparison_df.iloc[best_mae_idx]['Model']} (${comparison_df.iloc[best_mae_idx]['Test_MAE']:,.2f})")
print(f"Best AIC: {comparison_df.iloc[best_aic_idx]['Model']} ({comparison_df.iloc[best_aic_idx]['AIC']:,.2f})")
print(f"Best BIC: {comparison_df.iloc[best_bic_idx]['Model']} ({comparison_df.iloc[best_bic_idx]['BIC']:,.2f})")

# Create predictions comparison dataframe
test_predictions_pd = test_predictions.select("SALARY", "prediction").toPandas()
test_predictions_pd.columns = ["Actual", "Linear_Predicted"]

test_predictions_poly_pd = test_predictions_poly.select("SALARY", "prediction").toPandas()
test_predictions_poly_pd.columns = ["Actual_Poly", "Polynomial_Predicted"]

test_predictions_rf_pd = test_predictions_rf.select("SALARY", "prediction").toPandas()
test_predictions_rf_pd.columns = ["Actual_RF", "RF_Predicted"]

predictions_df = pd.concat([
    test_predictions_pd,
    test_predictions_poly_pd["Polynomial_Predicted"],
    test_predictions_rf_pd["RF_Predicted"]
], axis=1)

# Create visualization
fig, axes = plt.subplots(2, 2, figsize=(14, 12))
fig.suptitle('Actual vs Predicted Salary - Model Comparison', fontsize=16, fontweight='bold')

# Linear Regression plot
sns.scatterplot(data=predictions_df, x="Actual", y="Linear_Predicted", alpha=0.6, ax=axes[0, 0])
axes[0, 0].plot([predictions_df["Actual"].min(), predictions_df["Actual"].max()],
                [predictions_df["Actual"].min(), predictions_df["Actual"].max()],
                'r--', lw=2, label='Perfect Prediction')
axes[0, 0].set_title(f'Linear Regression\nR²={test_r2:.4f}, RMSE=${test_rmse:,.0f}')
axes[0, 0].set_xlabel('Actual Salary ($)')
axes[0, 0].set_ylabel('Predicted Salary ($)')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Polynomial Regression plot
sns.scatterplot(data=predictions_df, x="Actual", y="Polynomial_Predicted", alpha=0.6, ax=axes[0, 1], color='green')
axes[0, 1].plot([predictions_df["Actual"].min(), predictions_df["Actual"].max()],
                [predictions_df["Actual"].min(), predictions_df["Actual"].max()],
                'r--', lw=2, label='Perfect Prediction')
axes[0, 1].set_title(f'Polynomial Regression\nR²={test_r2_poly:.4f}, RMSE=${test_rmse_poly:,.0f}')
axes[0, 1].set_xlabel('Actual Salary ($)')
axes[0, 1].set_ylabel('Predicted Salary ($)')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Random Forest plot
sns.scatterplot(data=predictions_df, x="Actual", y="RF_Predicted", alpha=0.6, ax=axes[1, 0], color='orange')
axes[1, 0].plot([predictions_df["Actual"].min(), predictions_df["Actual"].max()],
                [predictions_df["Actual"].min(), predictions_df["Actual"].max()],
                'r--', lw=2, label='Perfect Prediction')
axes[1, 0].set_title(f'Random Forest\nR²={test_r2_rf:.4f}, RMSE=${test_rmse_rf:,.0f}')
axes[1, 0].set_xlabel('Actual Salary ($)')
axes[1, 0].set_ylabel('Predicted Salary ($)')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Residuals distribution
residuals_data = pd.DataFrame({
    'Linear': predictions_df["Actual"] - predictions_df["Linear_Predicted"],
    'Polynomial': predictions_df["Actual"] - predictions_df["Polynomial_Predicted"],
    'Random Forest': predictions_df["Actual"] - predictions_df["RF_Predicted"]
})

residuals_melted = residuals_data.melt(var_name='Model', value_name='Residual')
sns.boxplot(data=residuals_melted, x='Model', y='Residual', ax=axes[1, 1])
axes[1, 1].axhline(y=0, color='r', linestyle='--', lw=2)
axes[1, 1].set_title('Residuals Distribution by Model')
axes[1, 1].set_xlabel('Model')
axes[1, 1].set_ylabel('Residual (Actual - Predicted)')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n" + "=" * 80)
print("INTERPRETATION OF MODEL COMPARISON")
print("=" * 80)

print("\n1. INFORMATION CRITERIA INTERPRETATION:")
print("   - AIC (Akaike Information Criterion):")
print("     * Measures model quality balancing fit and complexity")
print("     * LOWER is better - penalizes adding unnecessary parameters")
print("     * Best for prediction-focused model selection")

print("\n   - BIC (Bayesian Information Criterion):")
print("     * Similar to AIC but penalizes complexity more strongly")
print("     * LOWER is better - stronger penalty for model complexity")
print("     * Best for identifying the 'true' model structure")

print("\n2. FINAL RECOMMENDATION:")
votes = {'Linear Regression': 0, 'Polynomial Regression': 0, 'Random Forest': 0}
votes[comparison_df.iloc[best_r2_idx]['Model']] += 1
votes[comparison_df.iloc[best_rmse_idx]['Model']] += 1
votes[comparison_df.iloc[best_aic_idx]['Model']] += 1
votes[comparison_df.iloc[best_bic_idx]['Model']] += 1

winner = max(votes, key=votes.get)
print(f"   Based on R², RMSE, AIC, and BIC: {winner} wins {votes[winner]}/4 metrics")

if winner == 'Linear Regression':
    print("\n   RECOMMENDATION: Use Linear Regression")
    print("   - Simplest model with competitive performance")
    print("   - Most interpretable coefficients")
elif winner == 'Polynomial Regression':
    print("\n   RECOMMENDATION: Use Polynomial Regression")
    print("   - Captures non-linear relationships")
    print("   - Still interpretable")
else:
    print("\n   RECOMMENDATION: Use Random Forest")
    print("   - Best predictive performance")
    print("   - Handles complex non-linear patterns")

print("\n   IMPORTANT: All models suffer from data leakage (SALARY_FROM/SALARY_TO)")
print("   Remove these features and retrain for realistic production deployment")

print("\n" + "=" * 80)